#### Create word embedding from corpus by Pretrained News vectors

In [1]:
from __future__ import unicode_literals, print_function
import gensim, logging, os
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import tokenize
import string
from itertools import chain

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
#word2vec.FAST_VERSION

In [3]:
content_dir = '../../Data/input/train/content'

class Corpus_Sentences(object):
    def __init__(self, dirname):
        self.dirname = dirname;
        
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
                
sentences = Corpus_Sentences(content_dir) # a memory-friendly iterator

In [4]:
# Load the pretrained glove vectors
glove_wv = KeyedVectors.load_word2vec_format('../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt', binary=False)

2018-05-02 23:15:34,584 : INFO : loading projection weights from ../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt
2018-05-02 23:17:50,116 : INFO : loaded (400000, 300) matrix from ../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt


In [ ]:
# Build vocabulary from corpus and words from pretrauned vector

model = Word2Vec(size=300, min_count=1, iter=10)
model.build_vocab(sentences)
training_examples_count = model.corpus_count
# below line will make it 1, so saving it before
model.build_vocab([list(glove_wv.vocab.keys())], update=True)

2018-05-02 23:17:50,566 : INFO : collecting all words and their counts
2018-05-02 23:17:50,923 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-02 23:18:02,140 : INFO : collected 18738 word types from a corpus of 325293 raw words and 854 sentences
2018-05-02 23:18:02,141 : INFO : Loading a fresh vocabulary
2018-05-02 23:18:02,226 : INFO : min_count=1 retains 18738 unique words (100% of original 18738, drops 0)
2018-05-02 23:18:02,227 : INFO : min_count=1 leaves 325293 word corpus (100% of original 325293, drops 0)
2018-05-02 23:18:02,459 : INFO : deleting the raw counts dictionary of 18738 items
2018-05-02 23:18:02,461 : INFO : sample=0.001 downsamples 39 most-common words
2018-05-02 23:18:02,465 : INFO : downsampling leaves estimated 236501 word corpus (72.7% of prior 325293)
2018-05-02 23:18:02,583 : INFO : estimated required memory for 18738 words and 300 dimensions: 54340200 bytes
2018-05-02 23:18:02,584 : INFO : resetting layer weights
2018-05-02 

In [ ]:
#if lockf=0.0 then pretrained word vectors will not be trained so keeping lockf=1.0
# Update the weights from pretrained vectors

model.intersect_word2vec_format("../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt",binary=False, lockf=1.0)

In [ ]:
# Train the model
model.train(sentences, total_examples=training_examples_count, epochs=100)

In [ ]:
#model.wv.save_word2vec_format('../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.bin')
model.wv.save_word2vec_format('../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt', binary=False)